In [6]:
import json
import os
import pandas as pd
import traceback
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [7]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
)